In [1]:
# import urllib2 as ul
import requests
import csv
# from lxml import html
from BeautifulSoup import BeautifulSoup as bs
import scipy.io as sio
from numpy import shape

MARKETBEAT_NASDAQ_URL = 'http://www.marketbeat.com/stocks/NASDAQ/{0}'
MARKETBEAT_NASDAQ_MOST_RECENT_URL = 'http://www.marketbeat.com/stocks/NASDAQ/{0}/?MostRecent=1'

### Here's a one-off for getting data on a single ticker

In [2]:
def get_ranking_csvrows_from_url(url,headers=None):
    try:
        page = requests.get(url)
    except Exception as inst:
        raise inst
    soup = bs(page.text)

    table = soup.find(lambda tag: tag.name == 'table' and
                                  tag.has_key('id') and
                                  tag['id'] == 'ratingstable' and
                                  tag.has_key('class') and 
                                  tag['class'] == "tablesorter")
    if table == None:
        raise Exception('No table in page')
        
    cur_headers = table.findAll('th') #,{'class':'header'})
    if cur_headers == None: 
        raise Exception('Headers not found')

    cur_headers = [ h.text.strip() for h in cur_headers ]
    cur_headers.insert(0,u'Ticker')

    if headers != None and headers != cur_headers:
        raise Exception('Wrong headers found', str(cur_headers))
    
    #csvout.writerow([u'Ticker'] + [ h.text for h in headers]) # concat 'Ticker' as the first column
        
#     %debug
    data = table.findAll('td') #,{'class':'yfnc_tabledata1'})
#     print shape(data)
    
    it = iter([ d.text.strip() for d in data ]) # create an iterator over the textual data
    csvrows = zip([tick]*(1+len(data)/5),it,it,it,it,it,it) # each call to it returns the next data entry, so this zip will create a 6-tuple array
#     print shape(csvrows)

    # dirty trick (only return 2 parameters if we don't know what we're looking for):
    if headers == None:
        return csvrows, cur_headers
    else:
        return csvrows

In [3]:
tick = 'INTC'
csvrows,headers = get_ranking_csvrows_from_url(MARKETBEAT_NASDAQ_URL.format(tick))

In [4]:
headers

[u'Ticker',
 u'Date',
 u'Firm',
 u'Action',
 u'Rating',
 u'Price Target',
 u'Actions']

In [5]:
MARKETBEAT_NASDAQ_URL.format(tick)

'http://www.marketbeat.com/stocks/NASDAQ/INTC'

In [6]:
# THIS SHOULD THROW AN EXCEPTION
tick = 'AAME'
try:
    csvrows = get_ranking_csvrows_from_url(MARKETBEAT_NASDAQ_URL.format(tick),headers)
except Exception as e:
    print 'SUCCESS'
    print e
else:
    print 'FAIL'

SUCCESS
('Wrong headers found', "[u'Ticker', u'Date', u'Quarter', u'Consensus Estimate', u'Actual EPS', u'Revenue Estimate', u'Actual Revenue', u'Release Link', u'Conference Call', u'Actions']")


In [7]:
print "\t".join(headers)
print "\t".join(csvrows[0])

Ticker	Date	Firm	Action	Rating	Price Target	Actions
INTC	2/2/2016	Goldman Sachs	Upgrade	Sell -> Neutral		&nbsp;&nbsp;&nbsp;&nbsp;


In [9]:
print shape(csvrows)
print shape(headers)
print "\t".join(csvrows[0])
print "\t".join(csvrows[1])

# the last cell is unneeded html code, we'll get rid of it later

(366, 7)
(7,)
INTC	2/2/2016	Goldman Sachs	Upgrade	Sell -> Neutral		&nbsp;&nbsp;&nbsp;&nbsp;
INTC	1/26/2016	Aegis	Reiterated Rating	Buy		&nbsp;&nbsp;&nbsp;&nbsp;


### And here's a loop for getting data on all the tickers from 'nasdaq.txt' and writing them to 'marketbeat_nasdaq.csv'

In [10]:
with open('NASDAQ.txt','r') as f:
    lines = f.readlines()

tickers = [l.strip() for l in lines if l.strip() != '']
# tickers

In [12]:
out = open('data/marketbeat_nasdaq.csv','a')
csvout = csv.writer(out)

headers = None
for i,tick in enumerate(tickers[1:]):
    tick = tick.split('\t')[0]
    print i,tick,',',
    try:
        if headers == None:
            csvrows,headers = get_ranking_csvrows_from_url(MARKETBEAT_NASDAQ_URL.format(tick))
        else:
            csvrows = get_ranking_csvrows_from_url(MARKETBEAT_NASDAQ_URL.format(tick),headers)
        csvrows = [ r[:-1] for r in csvrows ] # get rid of the last element in every tuple
    except Exception as inst:
        print inst
        continue

    csvout.writerows(csvrows)

out.flush()
out.close()
# csvout.close()        
# close(out)

 0 AAIT , No table in page
1 AAL , 2 AAME , ('Wrong headers found', "[u'Ticker', u'Date', u'Quarter', u'Consensus Estimate', u'Actual EPS', u'Revenue Estimate', u'Actual Revenue', u'Release Link', u'Conference Call', u'Actions']")
3 AAOI , 4 AAON , 5 AAPC , No table in page
6 AAPL , 7 AAVL , 8 AAWW , 9 AAXJ , ('Wrong headers found', "[u'Ticker', u'Announced', u'Period', u'Amount', u'Yield', u'Ex-Dividend Date', u'Record Date', u'Payable Date', u'Share']")
10 ABAC , ('Wrong headers found', "[u'Ticker', u'Date', u'Quarter', u'Consensus Estimate', u'Actual EPS', u'Revenue Estimate', u'Actual Revenue', u'Release Link', u'Conference Call', u'Actions']")
11 ABAX , 12 ABCB , 13 ABCD , 14 ABCO , 15 ABCW , 16 ABDC , 17 ABGB , 18 ABIO , 19 ABMD , 20 ABTL , 21 ABY , 22 ACAD , 23 ACAS , 24 ACAT , 25 ACET , 26 ACFC , 27 ACFN , 28 ACGL , 29 ACHC , 30 ACHN , 31 ACIW , 32 ACLS , 33 ACNB , ('Wrong headers found', "[u'Ticker', u'Announced', u'Period', u'Amount', u'Yield', u'Ex-Dividend Date', u'Record D

In [65]:
with open('stocks.csv','r') as infile:
    csvin = csv.reader(infile)
    data = map(tuple,csvin)

In [70]:
data[len(data)-1]

('ZX', '21-Jun-11', 'Oppenheimer', 'Initiated', '', 'Outperform')

In [72]:
sio.savemat('stocks.mat',{'data':data},do_compression=True)